In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
import cv2
import numpy as np
import tensorflow as tf
import random

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
base_url = './crime_data/Anomaly-Videos-Part-1/'
classes = [i for i in os.listdir(base_url) if not i.startswith('.')]
print(classes)

['RoadAccidents', 'Arson', 'Shoplifting', 'Stealing', 'Burglary', 'Fighting', 'Vandalism', 'Explosion', 'Normal_Videos_event', 'Arrest', 'Abuse', 'Robbery', 'Assault', 'Shooting']


In [4]:
class_ids = {}
id = 1
for c in classes:
    class_ids[c] = id
    id += 1

print(class_ids)

{'RoadAccidents': 1, 'Arson': 2, 'Shoplifting': 3, 'Stealing': 4, 'Burglary': 5, 'Fighting': 6, 'Vandalism': 7, 'Explosion': 8, 'Normal_Videos_event': 9, 'Arrest': 10, 'Abuse': 11, 'Robbery': 12, 'Assault': 13, 'Shooting': 14}


In [5]:
train_urls = open('./crime_data/UCF_Crimes-Train-Test-Split/Action_Regnition_splits/train_001.txt','r').read().split('\n')
len(train_urls)

532

In [6]:
random.shuffle(train_urls)

In [7]:
val_urls = [ train_urls.pop(random.randint(0,len(train_urls)-1)) for _ in range(int(0.20 * len(train_urls)))]
print('train length: ',len(train_urls))
print('val length: ', len(val_urls))

train length:  426
val length:  106


In [8]:
test_urls = open('./crime_data/UCF_Crimes-Train-Test-Split/Action_Regnition_splits/test_001.txt','r').read().split('\n')
len(test_urls)

169

In [9]:
x=train_urls[0].split('/')[0]
class_ids[x]

13

In [10]:
def format_frames(frame):
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.random_crop(frame, (220,220,3))
    return frame

def frames_from_video_file(video_path, n_frames):
  result = []
  src = cv2.VideoCapture(str(video_path))  
  start = random.randint(0,src.get(cv2.CAP_PROP_FRAME_COUNT)-(2*n_frames))
  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  frame_step = int(((src.get(cv2.CAP_PROP_FRAME_COUNT)-start) // n_frames) - 1)

  ret, frame = src.read()
  result.append(format_frames(frame))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

class FrameGenerator:
  def __init__(self,paths, n_frames,testing = False):
    self.n_frames = n_frames
    self.paths = paths
    self.testing = testing

  def get_files_and_class_names(self):
    video_paths = []
    labels = []
    for path in self.paths:
      video_paths.append(f'{base_url}{path}'.strip())
      labels.append(path.split('/')[0])
    pairs = list(zip(video_paths,labels))
    return pairs

  def __call__(self):
    pairs = self.get_files_and_class_names()

    if not self.testing:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames)
      label = class_ids[name]
      yield video_frames, label

In [11]:
nframes = 30
batch_size = 3
output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(train_urls,nframes),output_signature=output_signature)
val_ds = tf.data.Dataset.from_generator(FrameGenerator(val_urls,nframes),output_signature=output_signature)
test_ds = tf.data.Dataset.from_generator(FrameGenerator(test_urls,nframes),output_signature=output_signature)
train_ds = train_ds.repeat().batch(batch_size)
val_ds = val_ds.repeat().batch(batch_size)
test_ds = test_ds.batch(batch_size)

In [12]:
from tensorflow import keras
from tensorflow.keras import layers
import einops

In [13]:
HEIGHT = 220
WIDTH = 240

In [14]:
class Conv2Plus1D(keras.layers.Layer):
  def __init__(self, filters, kernel_size, padding):
    """
      A sequence of convolutional layers that first apply the convolution operation over the
      spatial dimensions, and then the temporal dimension. 
    """
    super().__init__()
    self.seq = keras.Sequential([  
        # Spatial decomposition
        layers.Conv3D(filters=filters,
                      kernel_size=(1, kernel_size[1], kernel_size[2]),
                      padding=padding),
        # Temporal decomposition
        layers.Conv3D(filters=filters, 
                      kernel_size=(kernel_size[0], 1, 1),
                      padding=padding)
        ])

  def call(self, x):
    return self.seq(x)

In [15]:
class ResidualMain(keras.layers.Layer):
  """
    Residual block of the model with convolution, layer normalization, and the
    activation function, ReLU.
  """
  def __init__(self, filters, kernel_size):
    super().__init__()
    self.seq = keras.Sequential([
        Conv2Plus1D(filters=filters,
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization(),
        layers.ReLU(),
        Conv2Plus1D(filters=filters, 
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization()
    ])

  def call(self, x):
    return self.seq(x)

In [16]:
class Project(keras.layers.Layer):
  """
    Project certain dimensions of the tensor as the data is passed through different 
    sized filters and downsampled. 
  """
  def __init__(self, units):
    super().__init__()
    self.seq = keras.Sequential([
        layers.Dense(units),
        layers.LayerNormalization()
    ])

  def call(self, x):
    return self.seq(x)

In [17]:
def add_residual_block(input, filters, kernel_size):
  """
    Add residual blocks to the model. If the last dimensions of the input data
    and filter size does not match, project it such that last dimension matches.
  """
  out = ResidualMain(filters, 
                     kernel_size)(input)

  res = input
  # Using the Keras functional APIs, project the last dimension of the tensor to
  # match the new filter size
  if out.shape[-1] != input.shape[-1]:
    res = Project(out.shape[-1])(res)

  return layers.add([res, out])

In [18]:
class ResizeVideo(keras.layers.Layer):
  def __init__(self, height, width):
    super().__init__()
    self.height = height
    self.width = width
    self.resizing_layer = layers.Resizing(height, width)

  def call(self, video):
    """
      Use the einops library to resize the tensor.  

      Args:
        video: Tensor representation of the video, in the form of a set of frames.

      Return:
        A downsampled size of the video according to the new height and width it should be resized to.
    """
    # b stands for batch size, t stands for time, h stands for height, 
    # w stands for width, and c stands for the number of channels.
    old_shape = einops.parse_shape(video, 'b t h w c')
    images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
    images = self.resizing_layer(images)
    videos = einops.rearrange(
        images, '(b t) h w c -> b t h w c',
        t = old_shape['t'])
    return videos

In [19]:
input_shape = (None, nframes, HEIGHT, WIDTH, 3)
input = layers.Input(shape=(input_shape[1:]))
x = input

x = Conv2Plus1D(filters=16, kernel_size=(3, 7, 7), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = ResizeVideo(HEIGHT // 2, WIDTH // 2)(x)

# Block 1
x = add_residual_block(x, 16, (3, 3, 3))
x = ResizeVideo(HEIGHT // 4, WIDTH // 4)(x)

# Block 2
x = add_residual_block(x, 32, (3, 3, 3))
x = ResizeVideo(HEIGHT // 8, WIDTH // 8)(x)

# Block 3
x = add_residual_block(x, 64, (3, 3, 3))
x = ResizeVideo(HEIGHT // 16, WIDTH // 16)(x)

# Block 4
x = add_residual_block(x, 128, (3, 3, 3))

x = layers.GlobalAveragePooling3D()(x)
x = layers.Flatten()(x)
x = layers.Dense(10)(x)

model = keras.Model(input, x)

In [20]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

estp = EarlyStopping(patience=10)
rlp = ReduceLROnPlateau(patience=3)

In [21]:
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              optimizer = keras.optimizers.Adam(), 
              metrics = ['accuracy'])

In [22]:
history = model.fit(x = train_ds,
                    epochs = 50, 
                    validation_data = val_ds,
                    callbacks = [estp, rlp],
                    steps_per_epoch=(len(train_urls)//batch_size),
                    validation_steps=(len(val_urls)//batch_size)
                    )

Epoch 1/50
142/142 ━━━━━━━━━━━━━━━━━━━━ 1026s 7s/step - accuracy: 0.0621 - loss: 1.5873 - val_accuracy: 0.0857 - val_loss: 1.5844 - learning_rate: 0.0010
Epoch 2/50
142/142 ━━━━━━━━━━━━━━━━━━━━ 1094s 8s/step - accuracy: 0.0745 - loss: 1.5978 - val_accuracy: 0.0952 - val_loss: 1.5694 - learning_rate: 0.0010
Epoch 3/50
142/142 ━━━━━━━━━━━━━━━━━━━━ 1166s 8s/step - accuracy: 0.0527 - loss: 1.6252 - val_accuracy: 0.0857 - val_loss: 1.6400 - learning_rate: 0.0010
Epoch 4/50


2024-04-02 22:12:00.963270: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Expected size[2] in [0, 2], but got 3
2024-04-02 22:12:01.025120: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: InvalidArgumentError: {{function_node __wrapped__Slice_device_/job:localhost/replica:0/task:0/device:GPU:0}} Expected size[2] in [0, 2], but got 3 [Op:Slice] name: 
Traceback (most recent call last):

  File "/Users/omar/Documents/GitHub/crime-anomaly-detection/.venv-metal-tf/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/Users/omar/Documents/GitHub/crime-anomaly-detection/.venv-metal-tf/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/omar/Documents/GitHub/crime-anomaly-detection/.venv-metal-tf/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator

  3/142 ━━━━━━━━━━━━━━━━━━━━ 21:48 9s/step - accuracy: 0.0370 - loss: 1.8967    

2024-04-02 22:12:25.678998: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Expected begin[0] in [0, 240], but got 15134196
2024-04-02 22:12:25.715635: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: InvalidArgumentError: {{function_node __wrapped__Slice_device_/job:localhost/replica:0/task:0/device:GPU:0}} Expected begin[0] in [0, 240], but got 15134196 [Op:Slice] name: 
Traceback (most recent call last):

  File "/Users/omar/Documents/GitHub/crime-anomaly-detection/.venv-metal-tf/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/Users/omar/Documents/GitHub/crime-anomaly-detection/.venv-metal-tf/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/omar/Documents/GitHub/crime-anomaly-detection/.venv-metal-tf/lib/python3.10/site-packages/tensorflow/python/dat

  5/142 ━━━━━━━━━━━━━━━━━━━━ 21:03 9s/step - accuracy: 0.0522 - loss: 1.7783 

KeyboardInterrupt: 

: 